In [44]:
from langchain_community.document_loaders import TextLoader, WebBaseLoader

loader = TextLoader('speech.txt')
text_documents = loader.load()
text_documents

[Document(page_content='Sometimes it\'s simply better to ignore the haters. That\'s the lesson that Tom\'s dad had been trying to teach him, but Tom still couldn\'t let it go. He latched onto them and their hate and couldn\'t let it go, but he also realized that this wasn\'t healthy. That\'s when he came up with his devious plan.\nThe choice was red, green, or blue. It didn\'t seem like an important choice when he was making it, but it was a choice nonetheless. Had he known the consequences at that time, he would likely have considered the choice a bit longer. In the end, he didn\'t and ended up choosing blue.\nHe stared out the window at the snowy field. He\'d been stuck in the house for close to a month and his only view of the outside world was through the window. There wasn\'t much to see. It was mostly just the field with an occasional bird or small animal who ventured into the field. As he continued to stare out the window, he wondered how much longer he\'d be shackled to the ste

In [45]:
# import bs4


# loader = WebBaseLoader(
#     web_paths=("https://en.wikipedia.org/wiki/Artificial_intelligence",),
#     bs_kwargs=dict(parse_only=bs4.SoupStrainer(["p", "div", "span"]))  # parse only these tags
# )

# text_documents = loader.load()

In [46]:
text_documents

[Document(page_content='Sometimes it\'s simply better to ignore the haters. That\'s the lesson that Tom\'s dad had been trying to teach him, but Tom still couldn\'t let it go. He latched onto them and their hate and couldn\'t let it go, but he also realized that this wasn\'t healthy. That\'s when he came up with his devious plan.\nThe choice was red, green, or blue. It didn\'t seem like an important choice when he was making it, but it was a choice nonetheless. Had he known the consequences at that time, he would likely have considered the choice a bit longer. In the end, he didn\'t and ended up choosing blue.\nHe stared out the window at the snowy field. He\'d been stuck in the house for close to a month and his only view of the outside world was through the window. There wasn\'t much to see. It was mostly just the field with an occasional bird or small animal who ventured into the field. As he continued to stare out the window, he wondered how much longer he\'d be shackled to the ste

In [47]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(text_documents)
documents[:5]

[Document(page_content="Sometimes it's simply better to ignore the haters. That's the lesson that Tom's dad had been trying to teach him, but Tom still couldn't let it go. He latched onto them and their hate and couldn't let it go, but he also realized that this wasn't healthy. That's when he came up with his devious plan.\nThe choice was red, green, or blue. It didn't seem like an important choice when he was making it, but it was a choice nonetheless. Had he known the consequences at that time, he would likely have considered the choice a bit longer. In the end, he didn't and ended up choosing blue.\nHe stared out the window at the snowy field. He'd been stuck in the house for close to a month and his only view of the outside world was through the window. There wasn't much to see. It was mostly just the field with an occasional bird or small animal who ventured into the field. As he continued to stare out the window, he wondered how much longer he'd be shackled to the steel bar insid

In [48]:
from dotenv import load_dotenv
import os
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
embeddings = OpenAIEmbeddings()

# Create FAISS vector store from documents
db = FAISS.from_documents(documents[:15], embeddings)


In [49]:
query = "What lesson was Tom’s dad trying to teach him"
docs = db.similarity_search(query)
docs[0].page_content

"Sometimes it's simply better to ignore the haters. That's the lesson that Tom's dad had been trying to teach him, but Tom still couldn't let it go. He latched onto them and their hate and couldn't let it go, but he also realized that this wasn't healthy. That's when he came up with his devious plan.\nThe choice was red, green, or blue. It didn't seem like an important choice when he was making it, but it was a choice nonetheless. Had he known the consequences at that time, he would likely have considered the choice a bit longer. In the end, he didn't and ended up choosing blue.\nHe stared out the window at the snowy field. He'd been stuck in the house for close to a month and his only view of the outside world was through the window. There wasn't much to see. It was mostly just the field with an occasional bird or small animal who ventured into the field. As he continued to stare out the window, he wondered how much longer he'd be shackled to the steel bar inside the house."

In [50]:
from langchain_openai import OpenAI

llm = OpenAI()

In [51]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    "Use the following pieces of context to answer the question at the end. "
    "If you don't know the answer, just say that you don't know, don't try to make up an answer. "
    "Context: {context} \n\n Question: {input}"
)

In [52]:
from langchain.chains.combine_documents import create_stuff_documents_chain
documents_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000284AE297770>)

In [55]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, documents_chain)

response = retrieval_chain.invoke({"input": query})

In [56]:
response['answer']

"?\n\nAnswer: The lesson that Tom's dad was trying to teach him was to ignore the haters and not let their hate affect him."